In [1]:
import numpy as np
import pandas as pd

In [176]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Bicycle_Rental_Prediction/train.csv', parse_dates=['date_time'], infer_datetime_format=True)
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Bicycle_Rental_Prediction/test.csv', parse_dates=['date_time'], infer_datetime_format=True)
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Bicycle_Rental_Prediction/sample_submission.csv')

In [177]:
all_data = pd.concat([train, test], ignore_index=True)
all_data_index = all_data.index

In [178]:
all_data.describe()

,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,number_of_rentals
count,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,273.000000
mean,198.634725,2.319503,0.121179,2.383970,58.386926,13.891959,23.356146,17.328552,59574.978022
std,56.845031,0.976251,0.230958,0.868715,12.396575,4.980942,4.969229,16.315020,27659.575774
min,44.123000,1.000000,0.000000,0.753000,24.831000,1.938000,9.895000,0.000000,1037.000000
25%,168.287250,1.405000,0.000000,1.743500,48.809000,10.172250,19.895000,4.357750,36761.000000
50%,205.729500,2.196000,0.000000,2.310500,58.784000,14.500000,24.000000,13.209500,63032.000000
75%,234.637250,3.143750,0.142000,2.800000,68.446000,18.000000,27.526000,25.608500,81515.000000
max,321.622000,4.000000,1.514000,5.607000,88.885000,22.312000,33.421000,82.162000,110377.000000


In [179]:
na_check = all_data.isna().sum() / len(all_data)
na_check

date_time                    0.00
wind_direction               0.00
sky_condition                0.00
precipitation_form           0.00
wind_speed                   0.00
humidity                     0.00
low_temp                     0.00
high_temp                    0.00
Precipitation_Probability    0.00
number_of_rentals            0.25
dtype: float64

In [180]:
for i in range(1, 9):
  all_data['wind_direc_'+(str)(i)] = False

for i in range(1, 9):
  all_data.loc[(all_data['wind_direction'] >= (45*(i-1))) & (all_data['wind_direction'] < (45*i)), 'wind_direc_'+(str)(i)] = True

In [181]:
all_data = all_data.drop(['wind_direction'], axis=1)

In [182]:
col_l = ['sky_condition', 'precipitation_form', 'wind_speed',
       'humidity', 'low_temp', 'high_temp', 'Precipitation_Probability']
col_r = all_data.columns.drop(col_l)

In [183]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaler.fit(all_data[col_l])
left_data = scaler.transform(all_data[col_l])
left_data = pd.DataFrame(left_data, columns=col, index=all_data_index)
left_data.describe()

,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability
count,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000
mean,0.439834,0.080039,0.336005,0.523869,0.586726,0.572182,0.210907
std,0.325417,0.152548,0.178969,0.193533,0.244475,0.211223,0.198571
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.135000,0.000000,0.204059,0.374340,0.404155,0.425062,0.053039
50%,0.398667,0.000000,0.320869,0.530068,0.616570,0.599549,0.160774
75%,0.714583,0.093791,0.421714,0.680910,0.788358,0.749426,0.311683
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [184]:
right_data = all_data[col_r]

right_data['year'] = right_data['date_time'].dt.year
right_data['month'] = right_data['date_time'].dt.month
right_data['day'] = right_data['date_time'].dt.day
right_data['wDay'] = right_data['date_time'].dt.weekday
right_data.index = all_data_index

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [185]:
all_data_ = pd.merge(left_data, right_data, left_index=True, right_index=True)#, on='id', how='outer')
#all_data_ = left_data.join(right_data, how='inner')

In [186]:
all_data = all_data_

In [187]:
nTrain = len(train)
train = all_data[:nTrain]
test = all_data[nTrain:]

In [190]:
all_data.columns

Index(['sky_condition', 'precipitation_form', 'wind_speed', 'humidity',
       'low_temp', 'high_temp', 'Precipitation_Probability', 'date_time',
       'number_of_rentals', 'wind_direc_1', 'wind_direc_2', 'wind_direc_3',
       'wind_direc_4', 'wind_direc_5', 'wind_direc_6', 'wind_direc_7',
       'wind_direc_8', 'year', 'month', 'day', 'wDay'],
      dtype='object')

In [194]:
col_x = ['sky_condition', 'precipitation_form', 'wind_speed', 'humidity',
       'low_temp', 'high_temp', 'Precipitation_Probability',
       'wind_direc_1', 'wind_direc_2', 'wind_direc_3',
       'wind_direc_4', 'wind_direc_5', 'wind_direc_6', 'wind_direc_7',
       'wind_direc_8', 'year', 'month', 'day', 'wDay']
col_y = ['number_of_rentals']

In [195]:
from sklearn.model_selection import train_test_split

train_x = train[col_x]
train_y = np.log1p(train['number_of_rentals'])

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, shuffle=True)

test_id_idx = test.index

test_x = test[col_x]

In [196]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(train_x, train_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [198]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
pred_train = lr.predict(train_x)
pred_val = lr.predict(val_x)

print('train mae score: ', mean_absolute_error(train_y, pred_train))
print('val mae score:', mean_absolute_error(val_y, pred_val))

# train mae score:  0.18014674463649302
# val mae score: 0.19129551489260352

train mae score:  0.18014674463649302
val mae score: 0.19129551489260352


In [199]:
pred = lr.predict(test_x)

In [207]:
submission[col_y] = np.expm1(pred)

In [208]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/Bicycle_Rental_Prediction/submission.csv', index=False)